In [20]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import sklearn
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
win = 50 #window size
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [21]:
#GET DATA
data = pd.read_csv('/content/drive/My Drive/hackathon/ground_truth/Subject39_1526591202/1526591202.csv',names=['values'])
data['values']=data['values'].apply(lambda x:x/1000.0)
arr = np.array(data['values'])
max_len = len(arr)
arr = np.reshape(arr,(1,max_len))
jp = pd.read_csv('/content/drive/My Drive/hackathon/ground_truth/Subject39_1526591202/jpeaks.csv',names=['values'])
jp['values'] = jp['values'].apply(lambda x:(x-1526591202)*226)
j_len = len(jp['values'])

In [22]:

prev_arr = np.ones((1,max_len))*-1 #position of previous Jpeak for each location

count = 0
val = -1

for i in range(0,max_len):
    if count + 1 != j_len:
        if jp['values'][count+1]>=i and jp['values'][count]<i:
            val = jp['values'][count]
            count = count + 1
    prev_arr[0][i]=val

In [23]:
def get_tuple():
    start = np.random.choice(range(0,max_len-win))
    x = arr[0][start:start+win]
    y = prev_arr[0][start+win-1]
    
    if y == prev_arr[0][start]:#if both start and end have same prev jpeak then no jpeak inbetween
        c = 0
        y = (y-start)/win
    else:
        c = 1
        y = (y-start)/win
    return x,y,c  

In [54]:
import random
#Generate Data
def generator(batch_size):
    while True:
        batchx = []
        batchy = []
        batchc = []
        count=0
        
        while count != batch_size:
            x,y,c = get_tuple()
            w = random.uniform(0,1)
            if (w<=0.72 and c==1) or (w>0.72 and c==0):#This is done to balance the two training classes
              batchx.append(x)
              batchy.append(y)
              batchc.append(c)
              count = count + 1
        
        batchx = tf.convert_to_tensor(batchx)
        batchy = tf.convert_to_tensor(batchy)
        batchc = tf.convert_to_tensor(batchc)

        batchx = tf.cast(batchx,dtype='float32')
        batchy = tf.cast(batchy,dtype='float32')
        batchc = tf.cast(batchc,dtype='float32')

        batchy = tf.reshape(batchy,[batch_size,1])
        batchc = tf.reshape(batchc,[batch_size,1])
        
        yield batchx,batchy,batchc

In [38]:
class mlp(layers.Layer):
  def __init__(self,units,drop):
    super(mlp,self).__init__()
    self.units = units
    self.drop = drop

    self.dense=[]
    self.drop_layer=layers.Dropout(self.drop)
    for unit in units:
      self.dense.append(layers.Dense(unit,activation=tf.nn.gelu))
  
  def get_config(self):
    config = super(mlp,self).get_config()
    config.update({"drop":self.drop,"units":self.units})
    return config
  
  def call(self,x):

    for i in range(len(self.units)):
      x = self.dense[i](x)
      x = self.drop_layer(x)
    return x


In [39]:
#Model Construction
leaky = layers.LeakyReLU(0.2)
inp = layers.Input((win,))
x = layers.LayerNormalization()(inp)
x = mlp([512,256,128],0.2)(x)
y = mlp([128,128],0.1)(x)
y=layers.Dense(1,activation='sigmoid')(y)
c = mlp([128,128],0.1)(x)
c=layers.Dense(1,activation='sigmoid')(c)
encoder = tf.keras.Model(inputs=inp,outputs=[y,c])

In [58]:
#Custom Training loop for model
class trainer(tf.keras.Model):


  def __init__(self,mod):
    super(trainer,self).__init__()
    self.encoder=mod

  def compile(self,optim):
    super(trainer,self).compile()
    self.optim=optim
    self.loss = tf.keras.metrics.Mean(name='loss')
    
  def train_step(self,data):

    x,y_true,c_true = data
        
    with tf.GradientTape() as tape:
            
        x,y_true,c_true = data
        y_preds,c_preds = self.encoder(x)
        
        z = tf.ones_like(y_true)
        reg_loss = tf.reduce_mean(tf.keras.losses.huber(y_true,y_preds)*c_true) 
        cls_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(c_true,c_preds))
            
        tot_loss = 0.1*reg_loss+cls_loss

    grads=tape.gradient(tot_loss,self.trainable_weights)
    self.optim.apply_gradients(zip(grads,self.trainable_weights))
    self.loss.update_state(tot_loss)
        
    return {'loss':self.loss.result()}
  @property
  def metrics(self):
    return [self.loss]


In [ ]:
#model compilation and training
final_model = trainer(encoder)
opt=tf.keras.optimizers.Adam(learning_rate=0.0001)
final_model.compile(opt)
final_model.fit(generator(64),steps_per_epoch=40,epochs=100,verbose=1)